<a href="https://colab.research.google.com/github/DockingBlade/Gene-Sequencing-/blob/main/Gene_Sequencing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing as sklpp
from sklearn import decomposition as skldecomp 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix


In [ ]:
df = pd.read_csv('splice.csv')

In [ ]:
numDataSamples = df.shape[0];
numAttributes = df.shape[1];

print('The number of Data Samples is ', numDataSamples);
print('The number of Attributes is ', numAttributes, ' but the Donor variable is not relevant to classification, since it is an identifier for the sample and does not have information relevant to the sample so there are really ', numAttributes-1);
print( 'attributes with 1 attribute as the Label or Dependent variable and ', numAttributes-2, ' independent variables, in this case nucleotides');



#Checing that all the nucleotide values are valid





The number of Data Samples is  3190
The number of Attributes is  62  but the Donor variable is not relevant to classification, since it is an identifier for the sample and does not have information relevant to the sample so there are really  61
attributes with 1 attribute as the Label or Dependent variable and  60  independent variables, in this case nucleotides


Exploration and Pre-processing: This Data set consists of sequences of 60 nucleotides, classified as introns, exons, or neither with a donor number. There are 3190 samples, with 765 being introns, and 762 being exons, and 1648 being neither. The nucleotides in the sequences are Adenine (A), Cytosine (C), Guanine (G), and Thymine(T). Since all variables are categorically, I will have to convert them to numeric values. I will use  label encoding to encode the nucleotides, and the labels. The dataset also contains some sequences that have ambigious nucleotide values, so I will remove this since I don't feel like they is an appropriate way to replace them. 



In [ ]:
#Pre-Processing

#Checing that all the nucleotide values are valid
for col in range(1,61):
    index = str(col);
    df.loc[((df[index] != 'A') & (df[index] != 'C') & (df[index] != 'G') & (df[index] != 'T')), index]= pd.NA

if df.isnull().values.any():
    print('Dataframe contains invalid values in the nucleotide sequences')
else:
    print('All entries in the nucleotide sequences are valid.')

df = df.dropna();

print('So in the dataset set description it mentions that nucleotides with abigious values were given different character values than A,G,C, and T. Thus I decided to remove these rows from the dataset, and now there are ',df.shape[0] , ' samples remaining');

index = 'Label';
df.loc[((df[index] != 'IE') & (df[index] != 'EI') & (df[index] != 'N') ), index]= pd.NA

if df.isnull().values.any():
    print('Dataframe contains invalid values in the Labels')
else:
    print('All entries in the Labels are now valid.')



#Label encoding the nucleotides
for col in range(1,61):
    index = str(col);
    df[index] = df[index].replace(['A','C','G','T'],[0, 1, 2, 3]);



df['Label'] = df['Label'].replace(['IE','EI','N'],[0, 1, 2]);



Dataframe contains invalid values in the nucleotide sequences
So in the dataset set description it mentions that nucleotides with abigious values were given different character values than A,G,C, and T. Thus I decided to remove these rows from the dataset, and now there are  3175  samples remaining
All entries in the Labels are now valid.


Only 15 samples have ambigious values in them, so I have removed them, and with the large number of samples removing 15 samples won't affect our training.

In [ ]:
#Feature Learning - PCA



nucleotides = df.iloc[:,np.r_[2:62]].to_numpy();
labels = df.iloc[:,np.r_[0]].to_numpy();

mean_datascaler = sklpp.StandardScaler(with_mean=True, with_std=False);
data_pca = skldecomp.PCA(n_components=0.95, svd_solver='full');

X_train, X_test, y_train, y_test = train_test_split(nucleotides, labels, test_size=0.20, shuffle=True);

X_train = mean_datascaler.fit_transform(X_train);
X_train =  data_pca.fit_transform(X_train);
mean = np.array(mean_datascaler.mean_);

test_X = X_test.astype(np.float64, copy=False)
for i in range(np.shape(test_X)[0]):
  test_X[i,:] -= mean;

U = np.transpose(data_pca.components_);
U_transpose = np.transpose(U);
Xtest_transpose = U_transpose @ np.transpose(test_X);
X_test = np.transpose(Xtest_transpose);

print('Shape of X_train,\n', X_train.shape);







Shape of X_train,
 (2540, 56)


Feature Learning :
Here I do PCA, to reduce the dimensonality of the data. I set the minimum amount of energy to capture to 95%, and it reduced the number of attribute from 60 to 56. I also split the data into training and test data, with 80% into training, and 20% test. I did PCA on the training data, extracted the mean, and then I took subtracted the mean from each row in the test data, and used the components calculated by PCA to project the test data to the lower dimension. 

In [ ]:
def get_score (model, train_input, train_label, test_input, test_label):
  
  model.fit(train_input,np.ravel(train_label));
  y_pred = model.predict(test_input);
  labels = np.ravel(test_label);
  
  return accuracy_score(labels,np.transpose(y_pred));

In [ ]:
skf = StratifiedKFold(n_splits=5);
skf.get_n_splits(X_train, y_train);
k = 1;
final = 1;
max = 0;


for train_index, test_index in skf.split(X_train, y_train):
  train_input, test_input = X_train[train_index], X_train[test_index]
  train_label, test_label = y_train[train_index], y_train[test_index]
  knn = KNeighborsClassifier(n_neighbors=k)
  val = get_score(knn, train_input, train_label, test_input, test_label)
  print('for k = ', k, ' the acurracy score on the validation set is ', val );
  if val > max:
    max = val
    final = k
  k+=2

print('Based on the acurracy scores on the validation sets we will choose', final, ' for the k parameter in KNN');

knn = KNeighborsClassifier(n_neighbors=final)
knn.fit(X_train, np.ravel(y_train));

y_pred = knn.predict(test_input);
labels = np.ravel(test_label);
  
print()
Recall = recall_score(labels,np.transpose(y_pred),average= 'micro');
print("Recall for KNN","\n", Recall );

Precision = precision_score(labels,np.transpose(y_pred),average='micro');
print("Precision for KNN","\n", Precision );

F_score = (2 * Recall * Precision)/(Recall + Precision);
print( "F score for KNN","\n", F_score);



print('The final accuracy is ',accuracy_score(labels,np.transpose(y_pred)));
print('The confusion matrix is \n', confusion_matrix(labels,np.transpose(y_pred), normalize='true'))

for k =  1  the acurracy score on the validation set is  0.6318897637795275
for k =  3  the acurracy score on the validation set is  0.6712598425196851
for k =  5  the acurracy score on the validation set is  0.6299212598425197
for k =  7  the acurracy score on the validation set is  0.6653543307086615
for k =  9  the acurracy score on the validation set is  0.6633858267716536
Based on the acurracy scores on the validation sets we will choose 3  for the k parameter in KNN

Recall for KNN 
 0.8267716535433071
Precision for KNN 
 0.8267716535433071
F score for KNN 
 0.8267716535433071
The final accuracy is  0.8267716535433071
The confusion matrix is 
 [[0.86885246 0.02459016 0.10655738]
 [0.07317073 0.86178862 0.06504065]
 [0.12547529 0.08365019 0.79087452]]


The first method we are using is KNN. I used cross-validation to determine which value of K to use. The algorithm determined k = 9 to be the best value, (at least in this run). 

In [ ]:
#logistic regression

logRegNoPenalty = LogisticRegression(penalty='none', solver='sag', max_iter= 1000, multi_class='ovr');
logRegL1 = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, multi_class='ovr');
logRegL2 = LogisticRegression(penalty='l2', solver='sag', max_iter=1000, multi_class='ovr');

skf = StratifiedKFold(n_splits=9);
skf.get_n_splits(X_train, y_train);
k = 0.1;
final = 0.1;
max = 0;





print();
for train_index, test_index in skf.split(X_train, y_train):
  train_input, test_input = X_train[train_index], X_train[test_index]
  train_label, test_label = y_train[train_index], y_train[test_index]
  logRegElasticNet = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=k, max_iter=1000,multi_class='ovr');
  val = get_score(logRegElasticNet, train_input, train_label, test_input, test_label)
  print('for lamda = ', k, ' the acurracy score on the validation set is ', val );
  if val > max:
    max = val
    final = k
  k+=0.1
print();
print('Based on the acurracy scores on the validation sets we will choose', final, ' for the lamda parameter in Elasticnet ');
print();

logRegNoPenalty.fit(X_train, np.ravel(y_train));
y_pred0 = logRegNoPenalty.predict(X_test);
labels = np.ravel(y_test);
noPenaltyAccuracy = accuracy_score(labels,np.transpose(y_pred0));
Recall0 = recall_score(labels,np.transpose(y_pred0),average= 'micro');
Precision0 = precision_score(labels,np.transpose(y_pred0),average='micro');
F_score0 = (2 * Recall * Precision)/(Recall + Precision);



logRegL1.fit(X_train, np.ravel(y_train));
y_pred1 = logRegL1.predict(X_test);
labels = np.ravel(y_test);
l1Accuracy = accuracy_score(labels,np.transpose(y_pred1));
Recall1 = recall_score(labels,np.transpose(y_pred1),average= 'micro');
Precision1 = precision_score(labels,np.transpose(y_pred1),average='micro');
F_score1 = (2 * Recall * Precision)/(Recall + Precision);

logRegL2.fit(X_train, np.ravel(y_train));
y_pred2 = logRegL2.predict(X_test);
labels = np.ravel(y_test);
l2Accuracy = accuracy_score(labels,np.transpose(y_pred2));
Recall2 = recall_score(labels,np.transpose(y_pred2),average= 'micro');
Precision2 = precision_score(labels,np.transpose(y_pred2),average='micro');
F_score2 = (2 * Recall * Precision)/(Recall + Precision);

logRegElasticNet = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=final, max_iter=1000, multi_class='ovr');
logRegElasticNet.fit(X_train, np.ravel(y_train));
y_pred3 = logRegElasticNet.predict(X_test);
labels = np.ravel(y_test);
elasticAccuracy = accuracy_score(labels,np.transpose(y_pred3));
Recall3 = recall_score(labels,np.transpose(y_pred3),average= 'micro');
Precision3 = precision_score(labels,np.transpose(y_pred3),average='micro');
F_score3 = (2 * Recall * Precision)/(Recall + Precision);



print('No Penalty accuracy is ', noPenaltyAccuracy);
print('The confusion matrix for no penalty is \n', confusion_matrix(labels,np.transpose(y_pred0), normalize='true'))
print("Recall for no penalty","\n", Recall0 );
print("Precision for no penalty","\n", Precision0 );
print( "F_score for no penalty","\n", F_score0);


print()
print('L1 Penalty accuracy is ', l1Accuracy);
print('The confusion matrix for L1 penalty is \n', confusion_matrix(labels,np.transpose(y_pred1), normalize='true'))
print("Recall for L1","\n", Recall1 );
print("Precision for L1","\n", Precision1 );
print("F_score for L1","\n", F_score1);

print()
print('L2 Penalty accuracy is ', l2Accuracy);
print('The confusion matrix for L2 penalty is \n', confusion_matrix(labels,np.transpose(y_pred2), normalize='true'))
print("Recall for L2","\n", Recall2 );
print("Precision for L2","\n", Precision2 );
print( "F_score for L2","\n", F_score2);

print()
print('Elastic Penalty accuracy is ', elasticAccuracy);
print('The confusion matrix for ElasticNet is \n', confusion_matrix(labels,np.transpose(y_pred3), normalize='true'))
print("Recall for Elastic","\n", Recall3 );
print("Precision for Elastic","\n", Precision3 );
print( "F_score for Elastic","\n", F_score3);

print()

if (noPenaltyAccuracy >= l1Accuracy) and (noPenaltyAccuracy >= l2Accuracy) and (noPenaltyAccuracy >= elasticAccuracy):
    print('We will choose the No Penalty Logistic Regression classifier because it has higher or equal accuracy then the other options for Logistic Regression');
elif (l1Accuracy >= noPenaltyAccuracy) and (l1Accuracy >= l2Accuracy) and (l1Accuracy >= elasticAccuracy):
    print('We will choose the L1 Penalty Logistic Regression classifier because it has higher or equal accuracy then the other options for Logistic Regression');
elif (l2Accuracy >= noPenaltyAccuracy) and (l2Accuracy >= l1Accuracy) and (l2Accuracy >= elasticAccuracy):
    print('We will choose the L2 Penalty Logistic Regression classifier because it has higher or equal accuracy then the other options for Logistic Regression');
elif (elasticAccuracy >= noPenaltyAccuracy) and (elasticAccuracy >= l2Accuracy) and (elasticAccuracy >= l1Accuracy):
    print('We will choose the Elasticnet Penalty Logistic Regression classifier because it has higher or equal accuracy then the other options for Logistic Regression');




for lamda =  0.1  the acurracy score on the validation set is  0.8091872791519434
for lamda =  0.2  the acurracy score on the validation set is  0.8021201413427562
for lamda =  0.30000000000000004  the acurracy score on the validation set is  0.8262411347517731
for lamda =  0.4  the acurracy score on the validation set is  0.8014184397163121
for lamda =  0.5  the acurracy score on the validation set is  0.8156028368794326
for lamda =  0.6  the acurracy score on the validation set is  0.8475177304964538
for lamda =  0.7  the acurracy score on the validation set is  0.8262411347517731
for lamda =  0.7999999999999999  the acurracy score on the validation set is  0.7730496453900709
for lamda =  0.8999999999999999  the acurracy score on the validation set is  0.8226950354609929

Based on the acurracy scores on the validation sets we will choose 0.6  for the lamda parameter in Elasticnet 

No Penalty accuracy is  0.8173228346456692
The confusion matrix for no penalty is 
 [[0.80645161 0.096

So Logistic Regression in Sklearn has a penalty option, with options for $||w||_1$ (L1 loss) , $||w||_2$ (L2 loss), and ElasticNet loss which is $\lambda *||w||_1$ + $(1-\lambda) * ||w||_2$, where $w$ is the parameter used in the $p(x)$ function, where $p(x) = 1/(1+e^{-(w^tx+b)}) $. The way that each of these are implemented in sklearn follows <br>
No Penality: $\displaystyle\min_{w,c}  C \sum_{n=1}^{\infty} \log(\exp(-y_i*(x_i^t*w + c)) + 1)$
<br>
L1 Penality: $\displaystyle\min_{w,c} \frac{1}{2} * ||w||_1 + C \sum_{i=1}^{n} \log(\exp(-y_i*(x_i^t*w + c)) + 1)$
<br>
L2 Penality: $\displaystyle\min_{w,c} \frac{1}{2} * ||w||_2^{2} + C \sum_{i=1}^{n} \log(\exp(-y_i*(x_i^t*w + c)) + 1)$
<br>
Elastic Penality: $\displaystyle\min_{w,c} \frac{1-\lambda}{2} * ||w||_2^{2} + \lambda * ||w||_1 + C \sum_{i=1}^{n} \log(\exp(-y_i*(x_i^t*w + c)) + 1)$

Note I have chosen to do one-vs-rest (ovr) classification since there are three classes here. SKlearn is training three sperate classifiers under the hood, and outputing the one that has the least loss as the label. 

Note that in the optimization there is a parameter C in each formulation, but I have left it at 1, the default value, because I am using a version of Stochastic Gradient Descent as the optimizer, and the C value only scales the derivative, but doesn't change its direction, so while it is likely to change the speed of convergence (faster if C is higher, slower if C is smaller, as this respectively increases/decreases the magnitude of the derivative), it isn't likely (in my opinon) to change the final accuracy too much.







In [ ]:
#QDA

QDA = QDA();
QDA.fit(X_train,np.ravel(y_train));

y_pred = QDA.predict(X_test);
labels = np.ravel(y_test);
Recall = recall_score(labels,np.transpose(y_pred),average= 'micro');
Precision = precision_score(labels,np.transpose(y_pred),average='micro');
F_score = (2 * Recall * Precision)/(Recall + Precision);

print('The accuracy for QDA is ', accuracy_score(labels,np.transpose(y_pred)));
print('The confusion matrix for QDA is \n', confusion_matrix(labels,np.transpose(y_pred), normalize='true'))
print("Recall for QDA","\n", Recall );
print("Precision for QDA","\n", Precision);
print("F_score for QDA","\n", F_score);




The accuracy for QDA is  0.8283464566929134
The confusion matrix for QDA is 
 [[0.70322581 0.06451613 0.23225806]
 [0.06       0.71333333 0.22666667]
 [0.04242424 0.01818182 0.93939394]]
Recall for QDA 
 0.8283464566929134
Precision for QDA 
 0.8283464566929134
F_score for QDA 
 0.8283464566929134


QDA has a closed form solution, so there isn't much to say here. 

Comparative analysis of the three methods on each dataset:
<br>
<br>


KNN :<br>
KNN accuracy is  0.8267716535433071 <br>
The confusion matrix for KNN is <br>
 [0.86885246 0.02459016 0.10655738] <br>
 [0.07317073 0.86178862 0.06504065] <br>
 [0.12547529 0.08365019 0.79087452] <br>
Recall for KNN <br>
 0.8267716535433071 <br>
Precision for KNN <br>
 0.8267716535433071 <br>
F score for KNN <br>
 0.8267716535433071 <br>


Logistic Regression:<br>
Logistic Penalty accuracy is  0.8188976377952756<br>
The confusion matrix for Logistic Regression penalty is <br>
 [0.80645161 0.09032258 0.10322581] <br>
 [0.06666667 0.79333333 0.14      ] <br>
 [0.08787879 0.07575758 0.83636364] <br>
Recall for Logistic Regression <br>
 0.8188976377952756 <br>
Precision for Logistic Regression <br>
 0.8188976377952756 <br>
F_score for Logistic Regression <br>
 0.8267716535433071 <br>

QDA : <br>
The accuracy for QDA is  0.8283464566929134
<br>
The confusion matrix for QDA is 
<br>
 [0.70322581 0.06451613 0.23225806]
 <br>
 [0.06       0.71333333 0.22666667]
 <br>
 [0.04242424 0.01818182 0.93939394]
 <br>
Recall for QDA <br>
 0.8283464566929134<br>
Precision for QDA <br>
 0.8283464566929134 <br>
F_score for QDA <br>
 0.8283464566929134 <br>



The time complexity of KNN in training is $O(1)$, while predicting is $O(n\log(k))$ However K is small so we can consider the prediction to be $O(n)$ time wise. 

The time complexity of training Logisitic Regression is $O((p+1)cnE)$ : 

p - number of features (+1 because of bias). Multiplication of each feature times it's weight (p operations, +1 for bias). Another p + 1 operations for summing all of them (obtaining prediction). Using gradient method to improve weights counts for the same number of operations, so in total we get 4* (p+1) (two for forward pass, two for backward), which is simply O(p+1).
<br>
c - number of classes (possible outputs) in your logistic regression.
<br>
n - number of samples in your dataset.
<br>
E - number of epochs you are willing to run the gradient descent (whole passes through dataset)

while the time complexity of predicting with Logisitic regression is $O((p+1)c)$

The time complexity of training QDA is at least $O(n*p^2)$ because of calculating the Covariance matrix 

The time complexity of predicting with QDA is either $O(p^2)$ or $O(p^3)$, depending on if the $det(Covariance)$ is pre-computed in the training 

QDA and KNN are very close in performance, but QDA has uneven performance. QDA has 0.70322581 accuracy on one class, and 0.71333333 on another class, and 0.93939394 on the last class. KNN has about 0.86, 0.86, and 0.79 on the three classes. Logistic Regression has accuracy of about 0.80, 0.79, and 0.84 on the classes, so its more even then the other two models. The three classifiers have similar Recall, and Precison, which means given a class, they are nearly equally likely to not label a mislabel an element from that class as one of the other two class (Recall), and equally likely to make a correct positive decision(Precision).


From an accuracy stand point, I believe KNN performs the best since it outperforms the other two classifiers on two of the three classes on a class-by-class basis (by a .1 margin on QDA), and outperforms Logisitc Regression on accuracy, while being 0.002 of QDA in terms of acccuracy.Thus my recommendation is to use KNN. However, assuming $det(Covariance)$ is pre-computed during training for QDA, KNN has the largest prediction time in terms of complexity. Thus, if time complexity was a concern (such as n very large), I would recomend logistic regression, since its total accuracy isn't too far from the other two's total accuracy, and it has the fasted time complexity (however in this case KNN computes quickly and thus has the advantage). 



Ethics:
<br>
I don't think that the ability to classify introns, and exons in itself. But the accuracy that can be acheived on this problem, indicates that Machine Learning can learn features from DNA. This implies it is possible to try to map DNA sequences to traits presented in people. This information could help doctors predict potentially diseases infants can have at birth. However insurance companies, might use this information to deny or increase the cost of services, adding to the already exorbitant cost of health services. 

Bibliography:
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
<br>
https://stackoverflow.com/questions/54238493/what-is-the-search-prediction-time-complexity-of-logistic-regression
<br>
https://medium.com/nerd-for-tech/why-a-time-complexity-for-training-a-k-nearest-neighbors-is-o-1-8a6041c50c7f
<br>
http://archive.ics.uci.edu/ml/datasets/Molecular+Biology+%28Splice-junction+Gene+Sequences%29